# QLoRA Tuning on Llama2

We will use the QLoRA technique to fine-tune the model in 4-bit precision and optimize VRAM usage.

Techniques applying:
- Quantization: HuggingFace Transformers has integrated optimum API to perform GPTQ quantization on LM. We can load and quantize our models in 8,4,3 or even 2 bits without a big drop of performance and still achieve faster inference speeds. This is achieved with the `BitsAndBytesConfig`. 
- LoRA: Stands for Low-rank Adaptation. It's widely used and effective for training custom LLMs. Read the paper [here](https://arxiv.org/abs/2305.14314).
- When you put quantization and LoRA together, we get QLoRA. Which, theoretically, reduces memory usage well.

In [9]:
import os
import pandas as pd
import creds

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer


In [17]:
## Prepare some variables
## model from HF hub
base_model_name = 'meta-llama/Llama-2-7b-hf'

## New Insturctional Dataset
instructional_dataset = 'mlabonne/guanaco-llama2-1k'

## Folder name to store finetuned model
folder_name = 'meta-llama2-7b-chat-guanaco'

### 1. Load the Dataset

In [18]:
dataset = load_dataset(instructional_dataset, split='train')

In [19]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [20]:
## view the dataset
print(pd.DataFrame(dataset).iloc[0]['text'])

<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>


In [21]:
print(len(dataset))

## This is good. Optimally, we want the instructional prompts to be ~1000.

1000


### 2. Prepare 4-bit quantization configuration

In [22]:
compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True, # data will be loaded in 4-bit format
    bnb_4bit_quant_type='nf4', # a quantizsation type
    bnb_4bit_compute_dtype=compute_dtype, # torch's float16
    bnb_4bit_use_double_quant=False # double quantization will not be used
)

### 3. Load Llama2 Model

In [23]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config, 
    device_map='auto', # automatically sets the device mapping
    token = creds.HUGGINGFACE_TOKEN
)

model.config.use_cache = False # disables the use of cache in the model config
model.config.pretraining_tp = 1 # sets the pretraining temperature parameter to 1 in the model config

config.json: 100%|██████████| 609/609 [00:00<00:00, 97.6kB/s]
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 4.84MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.98G/9.98G [01:27<00:00, 114MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 3.50G/3.50G [00:30<00:00, 114MB/s]
generation_config.json: 100%|██████████| 188/188 [00:00<00:00, 164kB/s]


### 4. Load Llama2 Tokenizer

In [25]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name,trust_remote_code=True,token=creds.HUGGINGFACE_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 586kB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 136MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 28.4MB/s]
special_tokens_map.json: 100%|██████████| 414/414 [00:00<00:00, 392kB/s]


### 5. Preparing the PEFT Parameters

In [26]:
peft_params = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

### 6. Training Parameters

In [29]:
training_params = TrainingArguments(
    output_dir="./results/meta-llama2-7b",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### 7. Model Fine-tuning

In [30]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text", # specifies the field in the dataset that contains text to be processed
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 9689.41 examples/s]


In [31]:
## train the model
trainer.train()

Step,Training Loss
25,1.215100
50,1.530300
75,1.188700
100,1.391100
125,1.158300
150,1.339200
175,1.154000
200,1.427600
225,1.126900
250,1.472000


TrainOutput(global_step=250, training_loss=1.3003206558227538, metrics={'train_runtime': 1076.9885, 'train_samples_per_second': 0.929, 'train_steps_per_second': 0.232, 'total_flos': 1.7122838399778816e+16, 'train_loss': 1.3003206558227538, 'epoch': 1.0})

In [48]:
## save the model and tokenizer
trainer.model.save_pretrained(folder_name, token = creds.HUGGINGFACE_TOKEN)
trainer.tokenizer.save_pretrained(folder_name)

GatedRepoError: 401 Client Error. (Request ID: Root=1-65e19f37-5619df085f20f80c387913f1;b46efd10-b827-4d78-bc53-0ca54d04bedb)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Repo model meta-llama/Llama-2-7b-hf is gated. You must be authenticated to access it.

In [31]:
## load the model you had saved
loaded_model = AutoModelForCausalLM.from_pretrained(f"./{folder_name}",
                                                    quantization_config = quant_config,
                                                    device_map='auto')
loaded_model.config.use_cache = False
loaded_model.config.pretraining_tp = 1

loaded_tokenizer = AutoTokenizer.from_pretrained(f"./{folder_name}", trust_remote_code=True)
loaded_tokenizer.pad_token = loaded_tokenizer.eos_token
loaded_tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.28s/it]


In [34]:
from tensorboard import notebook

log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

Reusing TensorBoard on port 4000 (pid 2840145), started 21:15:05 ago. (Use '!kill 2840145' to kill it.)

### 8. Inferencing

In [32]:
## Inference with SAVED MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe_loaded = pipeline(task="text-generation", model=loaded_model, tokenizer=loaded_tokenizer, max_length=200)
result_loaded = pipe_loaded(f"<s>[INST] {prompt} [/INST]")
print(result_loaded[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci (1452-1519) was an Italian polymath, artist, inventor, and engineer.

Da Vinci is widely considered one of the greatest painters of all time, and his most famous works include the Mona Lisa and The Last Supper. He was also a prolific inventor and engineer, and his designs for machines and devices were far ahead of his time.

Da Vinci was born in the town of Vinci, Italy, and he spent much of his life in Florence, where he was part of the city's artistic and intellectual circles. He was known for his curiosity and his ability to think creatively, and he was fascinated by the natural world and the way that things worked.

Da Vinci's legacy extends far beyond his art and inventions. He is often seen as


In [30]:
## Inference with IN-NOTEBOOK MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci (1452-1519) was an Italian polymath, artist, inventor, and engineer. He is widely considered one of the greatest painters of all time, and his inventions and designs were centuries ahead of his time. He is known for his famous works such as the Mona Lisa and The Last Supper, as well as his contributions to engineering, anatomy, and mathematics. Da Vinci was a true Renaissance man, and his legacy continues to inspire and influence people around the world.


In [37]:
## Inference with IN-NOTEBOOK MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci was an Italian Renaissance polymath: painter, sculptor, architect, musician, mathematician, engineer, inventor, anatomist, geologist, cartographer, botanist, and writer. 15 of his paintings have survived. He is regarded as one of the greatest painters of all time. 2 of his most famous paintings are The Mona Lisa and The Last Supper. 500 years after his death, he is still one of the most famous painters in the world. 13 of his works are in the Louvre. 12 of his works are in the Uffizi Gallery. 11 of his works are in the National Gallery. 9 of his works are in the Prado Museum. 8 of his works are in the Pinacoteca Ambrosiana. 6 of his


In [42]:
## load foundational model and tokenizer
foundational_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map='auto',
    token = creds.HUGGINGFACE_TOKEN
)
foundational_model.config.use_cache = False
foundational_model.config.pretraining_tp = 1

foundational_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True,token = creds.HUGGINGFACE_TOKEN)
foundational_tokenizer.pad_token = foundational_tokenizer.eos_token
foundational_tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


In [45]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "Who is Leonardo Da Vinci?"
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]
 nobody has the same handwriting as anyone else.
[INST] Who is Leonardo Da Vinci?
[INST] Who is Leonardo Da Vinci? [/INST]
nobody has the same handwriting as anyone else.
[INST] Who is Leonardo Da Vinci? [/INST]
[INST] Who is Leonardo Da Vinci? [/INST] nobody has the same handwriting as anyone else.
[INST] Who is Leonardo Da Vinci? [/INST] nobody has the same handwriting as anyone else.
nobody has the same handwriting as anyone else. nobody has the same handwriting as anyone else.
nobody has the same handwriting as anyone else. nobody has the same handwriting as anyone else. nobody has the same handwriting as anyone else.
nobody has the same hand


In [33]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "Who is Leonardo Da Vinci?"
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST]  Leonardo da Vinci (1452-1519) was a true Renaissance man, a polymath who excelled in various fields, including art, science, engineering, mathematics, and anatomy. everybody knows him as the most famous artist of the Italian Renaissance, but he was also a prolific inventor, engineer, and scientist. Here are some key facts about Leonardo da Vinci:

1. Early Life: Leonardo was born in Vinci, Italy, on April 15, 1452. His father, Messer Piero Fruosini, was a notary, and his mother, Caterina Buti, was a peasant.
2. Artistic Career: Leonardo began his artistic career as a young man in Florence, where he was apprenticed to the artist Andrea del Ver
